In [137]:
### 버전관리

In [138]:
!nvidia-smi

Wed Dec  3 13:30:09 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.10              Driver Version: 535.86.10    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090        On  | 00000000:4C:00.0 Off |                  N/A |
| 40%   31C    P8              18W / 350W |  22044MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
import pandas as pd
import os
import re
import json
import yaml
from glob import glob
from tqdm import tqdm
from pprint import pprint
import torch
import pytorch_lightning as pl
from rouge import Rouge # 모델의 성능을 평가하기 위한 라이브러리입니다.

from torch.utils.data import Dataset , DataLoader
from transformers import AutoTokenizer, BartForConditionalGeneration, BartConfig
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

from pathlib import Path
from transformers import BartForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datetime import datetime

import wandb
wandb.login(key="")                     # API KEY 입력

# # wandb 꺼두기
# os.environ["WANDB_DISABLED"] = "true"

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /data/ephemeral/home/.netrc


True

### Config 정의

In [140]:
# config 설정에 tokenizer 모듈이 사용되므로 미리 tokenizer를 정의해줍니다.
tokenizer = AutoTokenizer.from_pretrained("digit82/kobart-summarization")

In [141]:
config_data = {
    "general": {
        "data_path": "../data/",
        "data_raw_path": "../data/raw/",                                       
        "model_name": "digit82/kobart-summarization",                           # 모델 이름
        "output_dir": "../outputs/checkpoints/"                                 
    },
    "tokenizer": {
        "encoder_max_len": 512,
        "decoder_max_len": 100,
        "bos_token": f"{tokenizer.bos_token}",
        "eos_token": f"{tokenizer.eos_token}",
        "special_tokens": ['#Person1#', '#Person2#', '#Person3#',
                           '#PhoneNumber#', '#Address#', '#PassportNumber#']    # 특정 단어들이 분해되지 않도록 하기 위함
    },
    "training": {
        "overwrite_output_dir": True,
        "num_train_epochs": 20,
        "learning_rate": 1e-5,
        "per_device_train_batch_size": 50,
        "per_device_eval_batch_size": 32,
        "warmup_ratio": 0.1,
        "weight_decay": 0.01,
        "lr_scheduler_type": 'cosine',
        "optim": 'adamw_torch',
        "gradient_accumulation_steps": 1,
        "evaluation_strategy": 'epoch',
        "save_strategy": 'epoch',
        "save_total_limit": 5,
        "fp16": True,
        "load_best_model_at_end": True,
        "seed": 42,
        "logging_dir": "./logs",
        "logging_strategy": "epoch",
        "predict_with_generate": True,
        "generation_max_length": 100,
        "do_train": True,
        "do_eval": True,
        "early_stopping_patience": 3,
        "early_stopping_threshold": 0.001,
        # "report_to": "none"  
        "report_to": "wandb"                                                        # wandb 사용 옵션
    },
    "wandb": {
        # "entity": "bubblekid43",                                                   
        "project": "NLP-private",                                                   
        "name": "preprocessed_v1"                                                       # 버전관리
    },
    "inference": {
        "ckt_path": "../outputs/checkpoints/",                                      
        "result_path": "../outputs/prediction/",
        "no_repeat_ngram_size": 2,
        "early_stopping": True,
        "generate_max_length": 100,
        "num_beams": 4,
        "batch_size" : 32,
        "remove_tokens": ['<usr>', f"{tokenizer.bos_token}",
                          f"{tokenizer.eos_token}", f"{tokenizer.pad_token}"]       # 불필요한 생성 토큰 정의
    },
}

In [142]:
# 모델의 구성 정보를 YAML 파일로 저장
config_path = "../configs/config_v1.yaml"                                            # 버전관리
with open(config_path, "w") as file:
    yaml.dump(config_data, file, allow_unicode=True)
print(f"💾 config 저장 완료 : {config_path}")

# 저장된 config 파일 불러와서 내용 확인
with open(config_path, "r") as file:
    loaded_config = yaml.safe_load(file)
print("\n🔍 config 구성 정보 확인")
print(loaded_config)

💾 config 저장 완료 : ../configs/config_v1.yaml

🔍 config 구성 정보 확인
{'general': {'data_path': '../data/', 'data_raw_path': '../data/raw/', 'model_name': 'digit82/kobart-summarization', 'output_dir': '../outputs/checkpoints/'}, 'inference': {'batch_size': 32, 'ckt_path': '../outputs/checkpoints/', 'early_stopping': True, 'generate_max_length': 100, 'no_repeat_ngram_size': 2, 'num_beams': 4, 'remove_tokens': ['<usr>', '<s>', '</s>', '<pad>'], 'result_path': '../outputs/prediction/'}, 'tokenizer': {'bos_token': '<s>', 'decoder_max_len': 100, 'encoder_max_len': 512, 'eos_token': '</s>', 'special_tokens': ['#Person1#', '#Person2#', '#Person3#', '#PhoneNumber#', '#Address#', '#PassportNumber#']}, 'training': {'do_eval': True, 'do_train': True, 'early_stopping_patience': 3, 'early_stopping_threshold': 0.001, 'evaluation_strategy': 'epoch', 'fp16': True, 'generation_max_length': 100, 'gradient_accumulation_steps': 1, 'learning_rate': 1e-05, 'load_best_model_at_end': True, 'logging_dir': './logs', 'l

### 데이터 로드

In [143]:
# 데이터 경로 정의
data_path = Path(loaded_config['general']['data_path'])

In [144]:
# train data
train_df = pd.read_csv(data_path / 'raw' / 'train.csv')
print('✅ train_df.head()')
print(train_df.head())
display(train_df.head())

✅ train_df.head()
     fname                                           dialogue  \
0  train_0  #Person1#: 안녕하세요, Mr. Smith. 저는 Dr. Hawkins입니다...   
1  train_1  #Person1#: 안녕하세요, Mrs. Parker. 잘 지내셨나요?\n#Pers...   
2  train_2  #Person1#: 저기요, 열쇠 세트 본 적 있어요?\n#Person2#: 어떤 ...   
3  train_3  #Person1#: 너 여자친구 있는 거 왜 말 안 했어?\n#Person2#: 미...   
4  train_4  #Person1#: 안녕, 오늘 너무 멋져 보이네요. 저랑 춤 한 곡 추실래요?\n...   

                                             summary      topic  
0  Mr. Smith는 Dr. Hawkins에게 건강검진을 받으러 와서, 매년 검진 필...       건강검진  
1  Mrs. Parker가 Ricky와 함께 백신 접종을 위해 방문하였고, Dr. Pe...      백신 접종  
2  #Person1#은 열쇠 세트를 잃어버리고 #Person2#에게 찾는 것을 도와달라...      열쇠 분실  
3  #Person1#은 #Person2#가 여자친구가 있고 결혼할 예정이라는 사실을 말...  여자친구와의 결혼  
4  Malik은 Wen과 Nikki에게 춤을 제안하고, Wen은 발을 밟는 것을 감수하...       춤 제안  


,fname,dialogue,summary,topic
0,train_0,"#Person1#: 안녕하세요, Mr. Smith. 저는 Dr. Hawkins입니다...","Mr. Smith는 Dr. Hawkins에게 건강검진을 받으러 와서, 매년 검진 필...",건강검진
1,train_1,"#Person1#: 안녕하세요, Mrs. Parker. 잘 지내셨나요?\n#Pers...","Mrs. Parker가 Ricky와 함께 백신 접종을 위해 방문하였고, Dr. Pe...",백신 접종
2,train_2,"#Person1#: 저기요, 열쇠 세트 본 적 있어요?\n#Person2#: 어떤 ...",#Person1#은 열쇠 세트를 잃어버리고 #Person2#에게 찾는 것을 도와달라...,열쇠 분실
3,train_3,#Person1#: 너 여자친구 있는 거 왜 말 안 했어?\n#Person2#: 미...,#Person1#은 #Person2#가 여자친구가 있고 결혼할 예정이라는 사실을 말...,여자친구와의 결혼
4,train_4,"#Person1#: 안녕, 오늘 너무 멋져 보이네요. 저랑 춤 한 곡 추실래요?\n...","Malik은 Wen과 Nikki에게 춤을 제안하고, Wen은 발을 밟는 것을 감수하...",춤 제안


In [145]:
# validation data
val_df = pd.read_csv(data_path / 'raw' / 'dev.csv')
print('✅ val_df.head()')
print(val_df.head())
display(val_df.head())

✅ val_df.head()
   fname                                           dialogue  \
0  dev_0  #Person1#: 안녕하세요, 오늘 기분이 어떠세요?\n#Person2#: 요즘 ...   
1  dev_1  #Person1#: 야 Jimmy, 오늘 좀 이따 운동하러 가자.\n#Person2...   
2  dev_2  #Person1#: 나 진짜 건강에 안 좋은 음식 좀 그만 먹어야겠어. \n#Per...   
3  dev_3  #Person1#: 너 UFO 믿어?\n#Person2#: 당연하지, 있는 거 아냐...   
4  dev_4  #Person1#: 오늘 학교 갔어?\n#Person2#: 당연히 갔지. 너는?\n...   

                                             summary      topic  
0  #Person2#는 숨쉬기 어려워합니다. 의사는 #Person2#에게 증상을 확인하...      의사 상담  
1   #Person1#는 Jimmy를 운동하러 초대하고 팔과 복근 운동을 하도록 설득합니다.      운동 계획  
2  #Person1#은 건강에 안 좋은 음식을 그만 먹기로 결심하고, #Person2#...     건강한 식단  
3  #Person2#는 UFO를 믿고 꿈에서 볼 수 있다고 한다. #Person1#은 ...   UFO와 외계인  
4  #Person1#은 오늘 학교에 가지 않았고, #Person2#는 내일 학교 대신 ...  학교와 주말 계획  


,fname,dialogue,summary,topic
0,dev_0,"#Person1#: 안녕하세요, 오늘 기분이 어떠세요?\n#Person2#: 요즘 ...",#Person2#는 숨쉬기 어려워합니다. 의사는 #Person2#에게 증상을 확인하...,의사 상담
1,dev_1,"#Person1#: 야 Jimmy, 오늘 좀 이따 운동하러 가자.\n#Person2...",#Person1#는 Jimmy를 운동하러 초대하고 팔과 복근 운동을 하도록 설득합니다.,운동 계획
2,dev_2,#Person1#: 나 진짜 건강에 안 좋은 음식 좀 그만 먹어야겠어. \n#Per...,"#Person1#은 건강에 안 좋은 음식을 그만 먹기로 결심하고, #Person2#...",건강한 식단
3,dev_3,"#Person1#: 너 UFO 믿어?\n#Person2#: 당연하지, 있는 거 아냐...",#Person2#는 UFO를 믿고 꿈에서 볼 수 있다고 한다. #Person1#은 ...,UFO와 외계인
4,dev_4,#Person1#: 오늘 학교 갔어?\n#Person2#: 당연히 갔지. 너는?\n...,"#Person1#은 오늘 학교에 가지 않았고, #Person2#는 내일 학교 대신 ...",학교와 주말 계획


In [146]:
# test data
test_df = pd.read_csv(data_path / 'raw' / 'test.csv')
print('✅ test_df:', test_df.shape)
print(test_df.head())
display(test_df.head())

✅ test_df: (499, 2)
    fname                                           dialogue
0  test_0  #Person1#: Ms. Dawson, 받아쓰기 좀 부탁드려야겠어요. \n#Per...
1  test_1  #Person1#: 드디어 왔네! 뭐가 이렇게 오래 걸렸어?\n#Person2#: ...
2  test_2  #Person1#: Kate, 여기서 일어난 일을 믿기 힘들 거야.\n#Person...
3  test_3  #Person1#: 생일 축하해, 이거 너를 위한 선물이야, Brian.\n#Per...
4  test_4  #Person1#: 이 올림픽 공원 정말 크다! \n#Person2#: 맞아. 지금...


,fname,dialogue
0,test_0,"#Person1#: Ms. Dawson, 받아쓰기 좀 부탁드려야겠어요. \n#Per..."
1,test_1,#Person1#: 드디어 왔네! 뭐가 이렇게 오래 걸렸어?\n#Person2#: ...
2,test_2,"#Person1#: Kate, 여기서 일어난 일을 믿기 힘들 거야.\n#Person..."
3,test_3,"#Person1#: 생일 축하해, 이거 너를 위한 선물이야, Brian.\n#Per..."
4,test_4,#Person1#: 이 올림픽 공원 정말 크다! \n#Person2#: 맞아. 지금...


### 데이터 가공 및 데이터셋 클래스 구축
>- csv file 을 불러와서 encoder 와 decoder의 입력형태로 가공하기
>- 가공된 데이터를 torch dataset class 로 구축하여 모델에 입력가능한 형태로 만들기

In [147]:
# ========== 전처리 유틸 함수들 ========== #

# 1. 화자 태그 1/2 정규화
    # normalize_speaker_tags()에서
    # 한 샘플 내에서 등장하는 #PersonX#들을
        # 1번째 등장 → #Person1#
        # 2번째 등장 → #Person2#
        # 그 이후 → 전부 #Person2#로 합치도록 설계
    # dialogue와 summary에 동일 매핑 적용

# 2. turn 구조 정리
    # 줄 단위(\n)를 유지하면서
    # 각 줄마다 strip / 공백 정규화만 수행
    # 줄 자체를 합치거나 순서를 바꾸지 않음

# 3. 공백/개행 정규화
# normalize_spaces_and_newlines()에서
# 줄 앞뒤 공백 제거, 중복 공백을 " " 한 칸으로 통일
# 완전 빈 줄 삭제

# 4. 특수문자 (..., !! 등) 정규화
# normalize_special_chars()에서
# ..., ...., ....... → ...
# !!, !!!! → !
# ??, ???? → ?
# ~~, ~~~ → ~

# 5. Tier1 노이즈 축약
# compress_tier1_noise()에서
# ㅋㅋㅋㅋ → ㅋㅋ, ㅎㅎㅎㅎ → ㅎㅎ
# ㅠㅠㅠ, ㅜㅜㅜ → ㅠㅠ
# 어어어어, 아아아, 으으으, 음음음 같은 건
    # 어 / 아 / 음 으로 축약

# 6. leading filler 제거
# remove_leading_filler_after_tag()에서
# 패턴: #PersonX#: 네, ... → #PersonX#: ...
# 네,, 아,, 응,, 음,, 어,, 그래, 모두 제거 대상

In [148]:
# 1) 특수문자 정규화 (..., !!, ??? 등)
def normalize_special_chars(text: str) -> str:
    if not isinstance(text, str):
        return text
    # ... 또는 ....... → ...
    text = re.sub(r"\.{3,}", "...", text)
    # !!!, !!!! → !
    text = re.sub(r"!{2,}", "!", text)
    # ???, ???? → ?
    text = re.sub(r"\?{2,}", "?", text)
    # ~~, ~~~ → ~
    text = re.sub(r"~{2,}", "~", text)
    return text

# 2) 공백/개행 정규화
def normalize_spaces_and_newlines(text: str) -> str:
    if not isinstance(text, str):
        return text
    # 개별 줄 처리: 양끝 공백 제거 + 중복 공백 제거
    lines = text.split("\n")
    new_lines = []
    for line in lines:
        line = line.strip()
        line = re.sub(r"\s{2,}", " ", line)  # 중복 공백 → 한 칸
        if line:  # 완전 빈 줄은 제거
            new_lines.append(line)
    return "\n".join(new_lines)

# 3) Tier1 노이즈 축약 (ㅋ,ㅎ,ㅠ,ㅜ, 어/아/음 반복 등)
def compress_tier1_noise(text: str) -> str:
    if not isinstance(text, str):
        return text

    # ㅋㅋㅋㅋ, ㅎㅎㅎㅎ → ㅋㅋ / ㅎㅎ
    text = re.sub(r"[ㅋ]{2,}", "ㅋㅋ", text)
    text = re.sub(r"[ㅎ]{2,}", "ㅎㅎ", text)

    # ㅠㅠㅠㅠ, ㅜㅜㅜ → ㅠㅠ
    text = re.sub(r"[ㅠㅜ]{2,}", "ㅠㅠ", text)

    # 어어어어, 아아아아, 으으으 → 어 / 아 / 음 으로 축약
    def _compress_filler_token(token: str) -> str:
        # 어/아/음/으 + 반복 + 약간의 특수문자
        if re.fullmatch(r"[아어으음]+[\.…!?~]*", token):
            # '음', '으' 계열은 '음'으로 통일
            if "음" in token or "으" in token:
                return "음"
            # '아'가 포함되어 있으면 '아', 아니면 '어'
            if "아" in token:
                return "아"
            return "어"
        return token

    tokens = text.split(" ")
    tokens = [_compress_filler_token(tok) for tok in tokens]
    text = " ".join(tokens)

    return text

# 4) 화자 태그 정규화 + 1/2로 매핑
SPEAKER_TAG_PATTERN = re.compile(r"^(#Person(\d+)#):")

def normalize_speaker_tags(dialogue: str, summary: str | None = None):
    """
    - dialogue를 줄 단위로 스캔하면서
    - 등장하는 화자 태그(#Person3#, #Person4# 등)를
      #Person1#, #Person2# 두 개로만 매핑
    - 같은 샘플 내에서는 dialogue/summary 모두 일관된 매핑 사용
    """
    if not isinstance(dialogue, str):
        return dialogue, summary

    # 1) dialogue에서 화자 등장 순서에 따라 매핑 생성
    lines = dialogue.split("\n")
    orig_to_norm = {}
    next_slot = 1  # 1 → #Person1#, 2 → #Person2#

    new_lines = []
    for line in lines:
        m = SPEAKER_TAG_PATTERN.match(line.strip())
        if m:
            full_tag = m.group(1)   # "#PersonX#"
            spk_id = m.group(2)     # "3", "4", ...

            if full_tag not in orig_to_norm:
                if next_slot == 1:
                    orig_to_norm[full_tag] = "#Person1#"
                    next_slot = 2
                elif next_slot == 2:
                    orig_to_norm[full_tag] = "#Person2#"
                    next_slot = 3
                else:
                    # 세 번째 이후 화자는 전부 #Person2#로 합치기
                    orig_to_norm[full_tag] = "#Person2#"

            norm_tag = orig_to_norm[full_tag]
            # "#PersonX#:" → "#PersonY#:"
            line = re.sub(r"^#Person\d+#", norm_tag, line.strip())
        new_lines.append(line)

    new_dialogue = "\n".join(new_lines)

    # 2) summary에도 동일 매핑 적용 (혹시 모를 #Person3# 등장 케이스 대비)
    new_summary = summary
    if isinstance(summary, str) and orig_to_norm:
        for full_tag, norm_tag in orig_to_norm.items():
            new_summary = new_summary.replace(full_tag, norm_tag)

    return new_dialogue, new_summary

# 5) leading filler 제거 (#PersonX#: 네, / 아, / 응, / 음 등)
LEADING_FILLERS = ("네", "아", "응", "음", "어", "그래")

def remove_leading_filler_after_tag(dialogue: str) -> str:
    if not isinstance(dialogue, str):
        return dialogue

    lines = dialogue.split("\n")
    new_lines = []
    for line in lines:
        # 패턴: "#PersonX#: 네, ..." → "#PersonX#: ..."
        pattern = r"^(#Person\d+#:\s*)(%s)[,]?\s*" % "|".join(LEADING_FILLERS)
        line = re.sub(pattern, r"\1", line)
        new_lines.append(line)

    return "\n".join(new_lines)

In [149]:
# 데이터셋을 데이터프레임으로 변환하고 인코더와 디코더의 입력을 생성
class Preprocess:
    def __init__(self,
            bos_token: str,
            eos_token: str,
        ) -> None:

        self.bos_token = bos_token
        self.eos_token = eos_token

    @staticmethod
    def make_set_as_df(file_path, is_train: bool = True):
        """
        기존 baseline과 동일한 역할:
        - train/dev: fname, dialogue, summary
        - test    : fname, dialogue
        """
        df = pd.read_csv(file_path)
        if is_train:
            df = df[['fname', 'dialogue', 'summary']]
        else:
            df = df[['fname', 'dialogue']]
        return df

    def preprocess_pair(self, dialogue: str, summary: str | None = None, is_test: bool = False):
        """
        Preprocess_v1 전체 적용:
        - 화자 태그 1/2 정규화
        - turn 구조 유지(줄 단위 처리)
        - 공백/개행 정규화
        - 특수문자 정규화
        - Tier1 노이즈 축약
        - leading filler 제거
        """

        # 1) 화자 태그 1/2 정규화 (dialogue & summary 동시에)
        dialogue, summary = normalize_speaker_tags(dialogue, summary)

        # 2) 공백/개행 정규화 (줄단위 strip + 빈줄 제거)
        dialogue = normalize_spaces_and_newlines(dialogue)
        if summary is not None:
            summary = normalize_spaces_and_newlines(summary)

        # 3) 특수문자 정규화 (.../!!/??/~~)
        dialogue = normalize_special_chars(dialogue)
        if summary is not None:
            summary = normalize_special_chars(summary)

        # 4) Tier1 노이즈 축약 (어/아/음, ㅋㅋ, ㅠㅠ 등)
        dialogue = compress_tier1_noise(dialogue)
        if summary is not None:
            summary = compress_tier1_noise(summary)

        # 5) leading filler 제거 (#PersonX#: 네, / 아, / 응, / 음 등)
        dialogue = remove_leading_filler_after_tag(dialogue)

        if is_test:
            return dialogue, None
        else:
            return dialogue, summary

    # BART 모델의 입력, 출력 형태를 맞추기 위해 전처리를 진행
    def make_input(self, dataset: pd.DataFrame, is_test: bool = False):
        """
        - dataset: make_set_as_df로 만든 df
        - is_test:
            - True  → dialogue만 있고 summary 없음
            - False → dialogue + summary 모두 있음
        """
        if is_test:
            encoder_input_list = []
            decoder_input_list = []

            for row in dataset.itertuples(index=False):
                clean_dialogue, _ = self.preprocess_pair(row.dialogue, None, is_test=True)
                encoder_input_list.append(clean_dialogue)
                # test에서는 디코더 입력에 bos_token만 사용
                decoder_input_list.append(self.bos_token)

            return encoder_input_list, decoder_input_list

        else:
            encoder_input_list = []
            decoder_input_list = []
            decoder_output_list = []

            for row in dataset.itertuples(index=False):
                clean_dialogue, clean_summary = self.preprocess_pair(row.dialogue, row.summary, is_test=False)

                # encoder input: 전처리된 dialogue
                encoder_input_list.append(clean_dialogue)

                # decoder input: bos + summary (teacher forcing)
                decoder_input_list.append(self.bos_token + str(clean_summary))

                # decoder output: summary + eos
                decoder_output_list.append(str(clean_summary) + self.eos_token)

            return encoder_input_list, decoder_input_list, decoder_output_list

In [ ]:
def save_preprocessed_csv(preprocessor, config, version="v1"):
    """
    Preprocess_v1 적용 → 새로운 CSV 파일로 저장하는 함수
    예:
        train_preprocessed_v1.csv
        dev_preprocessed_v1.csv
        test_preprocessed_v1.csv
    """
    data_path = config['general']['data_path']

    # --- 파일 경로 정의 ---
    train_path = f"{data_path}/raw/train.csv"
    dev_path = f"{data_path}/raw/dev.csv"
    test_path = f"{data_path}/raw/test.csv"

    # --- 데이터 로드 ---
    train_df = pd.read_csv(train_path)
    dev_df = pd.read_csv(dev_path)
    test_df = pd.read_csv(test_path)

    # 새 데이터프레임 만들기
    new_train = []
    new_dev = []
    new_test = []

    # ===== Train 전처리 =====
    for row in train_df.itertuples(index=False):
        clean_dialogue, clean_summary = preprocessor.preprocess_pair(
            row.dialogue,
            row.summary,
            is_test=False
        )
        new_train.append([row.fname, clean_dialogue, clean_summary, row.topic])

    # ===== Dev 전처리 =====
    for row in dev_df.itertuples(index=False):
        clean_dialogue, clean_summary = preprocessor.preprocess_pair(
            row.dialogue,
            row.summary,
            is_test=False
        )
        new_dev.append([row.fname, clean_dialogue, clean_summary, row.topic])

    # ===== Test 전처리 =====
    for row in test_df.itertuples(index=False):
        clean_dialogue, _ = preprocessor.preprocess_pair(
            row.dialogue,
            None,
            is_test=True
        )
        new_test.append([row.fname, clean_dialogue])

    # ===== pandas DataFrame 변환 =====
    new_train_df = pd.DataFrame(new_train, columns=["fname", "dialogue", "summary", "topic"])
    new_dev_df = pd.DataFrame(new_dev, columns=["fname", "dialogue", "summary", "topic"])
    new_test_df = pd.DataFrame(new_test, columns=["fname", "dialogue"])

    # ===== 파일로 저장 =====
    save_dir = f"{data_path}/processed/preprocessed_{version}"
    os.makedirs(save_dir, exist_ok=True)

    new_train_df.to_csv(f"{save_dir}/train_preprocessed_{version}.csv", index=False)
    new_dev_df.to_csv(f"{save_dir}/dev_preprocessed_{version}.csv", index=False)
    new_test_df.to_csv(f"{save_dir}/test_preprocessed_{version}.csv", index=False)

    print("✅ 전처리 완료!")
    print(f"💾 전처리된 데이터 저장 완료 | 저장 위치: {save_dir}")

In [151]:
# Train에 사용되는 Dataset 클래스를 정의
class DatasetForTrain(Dataset):
    def __init__(self, encoder_input, decoder_input, labels, len):
        self.encoder_input = encoder_input
        self.decoder_input = decoder_input
        self.labels = labels
        self.len = len

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()}      # item[input_ids], item[attention_mask]
        item2 = {key: val[idx].clone().detach() for key, val in self.decoder_input.items()}     # item2[input_ids], item2[attention_mask]
        item2['decoder_input_ids'] = item2['input_ids']
        item2['decoder_attention_mask'] = item2['attention_mask']
        item2.pop('input_ids')
        item2.pop('attention_mask')
        # item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask]
        item.update(item2)
        # item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask], item[labels]                                                                  
        item['labels'] = self.labels['input_ids'][idx]
        return item

    def __len__(self):
        return self.len

# Validation에 사용되는 Dataset 클래스를 정의
class DatasetForVal(Dataset):
    def __init__(self, encoder_input, decoder_input, labels, len):
        self.encoder_input = encoder_input
        self.decoder_input = decoder_input
        self.labels = labels
        self.len = len

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()}      # item[input_ids], item[attention_mask]
        item2 = {key: val[idx].clone().detach() for key, val in self.decoder_input.items()}     # item2[input_ids], item2[attention_mask]
        item2['decoder_input_ids'] = item2['input_ids']
        item2['decoder_attention_mask'] = item2['attention_mask']
        item2.pop('input_ids')
        item2.pop('attention_mask')
        # item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask]
        item.update(item2)
        # item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask], item[labels]
        item['labels'] = self.labels['input_ids'][idx]
        return item

    def __len__(self):
        return self.len

# Test에 사용되는 Dataset 클래스를 정의
class DatasetForInference(Dataset):
    def __init__(self, encoder_input, test_id, len):
        self.encoder_input = encoder_input
        self.test_id = test_id
        self.len = len

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()}
        item['ID'] = self.test_id[idx]
        return item

    def __len__(self):
        return self.len

In [152]:
# tokenization 과정까지 진행된 최종적으로 모델에 입력될 데이터를 출력
def prepare_train_dataset(config, preprocessor, data_path, tokenizer):
    train_file_path = os.path.join(data_path, 'raw', 'train.csv')
    val_file_path = os.path.join(data_path, 'raw', 'dev.csv')

    # train, validation에 대해 각각 데이터프레임을 구축
    train_data = preprocessor.make_set_as_df(train_file_path)
    val_data = preprocessor.make_set_as_df(val_file_path)

    # print('\n🚀 train, valid 데이터 로드 중...')
    encoder_input_train , decoder_input_train, decoder_output_train = preprocessor.make_input(train_data)
    encoder_input_val , decoder_input_val, decoder_output_val = preprocessor.make_input(val_data)
    print("\n✅ 학습 데이터 로드 완료!")

    # print('\n🚀 train, valid 데이터셋 생성 중...')
    tokenized_encoder_inputs = tokenizer(encoder_input_train, return_tensors="pt", padding=True,
                            add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False)
    tokenized_decoder_inputs = tokenizer(decoder_input_train, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)
    tokenized_decoder_ouputs = tokenizer(decoder_output_train, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)

    train_inputs_dataset = DatasetForTrain(tokenized_encoder_inputs, tokenized_decoder_inputs, tokenized_decoder_ouputs,len(encoder_input_train))

    val_tokenized_encoder_inputs = tokenizer(encoder_input_val, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False)
    val_tokenized_decoder_inputs = tokenizer(decoder_input_val, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)
    val_tokenized_decoder_ouputs = tokenizer(decoder_output_val, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)

    val_inputs_dataset = DatasetForVal(val_tokenized_encoder_inputs, val_tokenized_decoder_inputs, val_tokenized_decoder_ouputs,len(encoder_input_val))
    print("\n✅ 학습 데이터셋 생성 완료!")

    return train_inputs_dataset, val_inputs_dataset

### Trainer 및 Trainingargs 구축하기
>- Huggingface 의 Trainer 와 Training arguments를 활용하여 모델 학습을 일괄적으로 처리해주는 클래스를 정의

In [153]:
# 모델 성능에 대한 평가 지표 정의
def compute_metrics(config,tokenizer,pred):
    rouge = Rouge()
    predictions = pred.predictions
    labels = pred.label_ids

    predictions[predictions == -100] = tokenizer.pad_token_id
    labels[labels == -100] = tokenizer.pad_token_id

    decoded_preds = tokenizer.batch_decode(predictions, clean_up_tokenization_spaces=True)
    labels = tokenizer.batch_decode(labels, clean_up_tokenization_spaces=True)

    # 정확한 평가를 위해 미리 정의된 불필요한 생성 토큰들을 제거
    replaced_predictions = decoded_preds.copy()
    replaced_labels = labels.copy()
    remove_tokens = config['inference']['remove_tokens']        # config에서 정의한 불필요한 생성 토큰
    for token in remove_tokens:
        replaced_predictions = [sentence.replace(token," ") for sentence in replaced_predictions]
        replaced_labels = [sentence.replace(token," ") for sentence in replaced_labels]

    print('\n🚀 불필요한 생성 토큰 제거 중...')
    print(f"PRED: {replaced_predictions[0]}")
    print(f"GOLD: {replaced_labels[0]}")
    # print('-'*150)
    # print(f"PRED: {replaced_predictions[1]}")
    # print(f"GOLD: {replaced_labels[1]}")
    # print('-'*150)
    # print(f"PRED: {replaced_predictions[2]}")
    # print(f"GOLD: {replaced_labels[2]}")
    print('\n✅ 불필요한 생성 토큰 제거 완료!')

    # 최종적인 ROUGE 점수를 계산
    results = rouge.get_scores(replaced_predictions, replaced_labels,avg=True)

    # ROUGE 점수 중 F-1 score를 통해 평가
    result = {key: value["f"] for key, value in results.items()}
    return result

In [154]:
# 학습을 위한 trainer 클래스와 매개변수를 정의
def load_trainer_for_train(config,generate_model,tokenizer,train_inputs_dataset,val_inputs_dataset):
    
    # print('\n🚀 training arguments 정의 중...')
    # set training args
    training_args = Seq2SeqTrainingArguments(
                output_dir=config['general']['output_dir'],
                overwrite_output_dir=config['training']['overwrite_output_dir'],
                num_train_epochs=config['training']['num_train_epochs'],
                learning_rate=config['training']['learning_rate'],
                per_device_train_batch_size=config['training']['per_device_train_batch_size'],
                per_device_eval_batch_size=config['training']['per_device_eval_batch_size'],
                warmup_ratio=config['training']['warmup_ratio'],
                weight_decay=config['training']['weight_decay'],
                lr_scheduler_type=config['training']['lr_scheduler_type'],
                optim =config['training']['optim'],
                gradient_accumulation_steps=config['training']['gradient_accumulation_steps'],
                evaluation_strategy=config['training']['evaluation_strategy'],
                save_strategy =config['training']['save_strategy'],
                save_total_limit=config['training']['save_total_limit'],
                fp16=config['training']['fp16'],
                load_best_model_at_end=config['training']['load_best_model_at_end'],
                seed=config['training']['seed'],
                logging_dir=config['training']['logging_dir'],
                logging_strategy=config['training']['logging_strategy'],
                predict_with_generate=config['training']['predict_with_generate'],
                generation_max_length=config['training']['generation_max_length'],
                do_train=config['training']['do_train'],
                do_eval=config['training']['do_eval'],
                report_to=config['training']['report_to']                             # wandb 사용 설정
            )

    # wandb 초기화
    wandb.init(
        # entity=config['wandb']['entity'],
        project=config['wandb']['project'],
        name=config['wandb']['name'],
        dir="/root/nlp-competition/wandb",
    )

    # wandb checkpoints 저장하는 환경 변수를 설정
    os.environ["WANDB_LOG_MODEL"]="true"
    os.environ["WANDB_WATCH"]="false"

    # EarlyStopping : Validation loss가 더 이상 개선되지 않을 때 학습을 중단
    MyCallback = EarlyStoppingCallback(
        early_stopping_patience=config['training']['early_stopping_patience'],
        early_stopping_threshold=config['training']['early_stopping_threshold']
    )
    print('\n✅ training arguments 정의 완료!')
    
    # print('\n🚀 trainer 생성 중...')
    # Trainer 클래스 정의
    trainer = Seq2SeqTrainer(
        model=generate_model,                                                           # 사용자가 사전 학습하기 위해 사용할 모델 입력
        args=training_args,
        train_dataset=train_inputs_dataset,
        eval_dataset=val_inputs_dataset,
        compute_metrics = lambda pred: compute_metrics(config,tokenizer, pred),
        callbacks = [MyCallback]
    )
    print('\n✅ trainer 생성 완료!')

    return trainer

In [155]:
# 학습을 위한 tokenizer와 사전 학습된 모델 불러오기
def load_tokenizer_and_model_for_train(config,device):

    # print('\n🚀 tokenizer & model 로드 중...')
    print(f'\n🤖 모델명 : {config["general"]["model_name"]}')
    model_name = config['general']['model_name']
    bart_config = BartConfig().from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    generate_model = BartForConditionalGeneration.from_pretrained(config['general']['model_name'],config=bart_config)

    special_tokens_dict={'additional_special_tokens':config['tokenizer']['special_tokens']}
    tokenizer.add_special_tokens(special_tokens_dict)

    generate_model.resize_token_embeddings(len(tokenizer))          # 사전에 special token을 추가했으므로 재구성
    generate_model.to(device)
    # print(generate_model.config)
    # print('✅ tokenizer & model 로드 완료!')

    return generate_model , tokenizer

In [ ]:
# 체크포인트 평가 함수
def eval_checkpoint(ckpt_path, config, tokenizer, eval_dataset):
    print(f"\n=== Evaluate {ckpt_path} ===")
    model = BartForConditionalGeneration.from_pretrained(ckpt_path)
    model.resize_token_embeddings(len(tokenizer))

    args = Seq2SeqTrainingArguments(
        output_dir="../outputs/tmp_eval",
        per_device_eval_batch_size=config["training"]["per_device_eval_batch_size"],
        predict_with_generate=True,
        do_train=False,
        do_eval=True,
        report_to="wandb",           # wandb 설정
    )

    trainer = Seq2SeqTrainer(
        model=model,
        args=args,
        eval_dataset=eval_dataset,
        compute_metrics=lambda pred: compute_metrics(config, tokenizer, pred),
    )

    metrics = trainer.evaluate()
    print(metrics)
    return metrics

In [157]:
# best & 마지막 checkpoint 찾는 함수
def get_best_and_last_checkpoints(config):
    output_dir = Path(config["general"]["output_dir"])
    ckpt_root = Path(config["inference"]["ckt_path"])

    # 마지막 checkpoint 찾기
    ckpt_dirs = [p for p in ckpt_root.glob("checkpoint-*") if p.is_dir()]
    def _get_step(p: Path) -> int:
        return int(p.name.split("-")[-1])
    ckpt_dirs = sorted(ckpt_dirs, key=_get_step)
    last_ckpt = ckpt_dirs[-1]

    # best checkpoint 찾기
    state_path = last_ckpt / "trainer_state.json"
    with open(state_path, "r") as f:
        state = json.load(f)
    best_ckpt = state.get("best_model_checkpoint", None)

    print(f"\n🚀 체크포인트 탐색 중...")
    print("- best checkpoint :", best_ckpt)
    print("- last_ckpt :", last_ckpt)

    return str(best_ckpt), str(last_ckpt)

In [158]:
def extract_exp_config(config: dict) -> dict:
    # 수정 필요
    return {
        "exp_name": config.get("wandb", {}).get("name", "no_name"),
        "model_name": config["general"]["model_name"],
        "num_train_epochs": config["training"]["num_train_epochs"],
        "learning_rate": config["training"]["learning_rate"],
        "per_device_train_batch_size": config["training"]["per_device_train_batch_size"],
        "per_device_eval_batch_size": config["training"]["per_device_eval_batch_size"],
        "warmup_ratio": config["training"]["warmup_ratio"],
        "weight_decay": config["training"]["weight_decay"],
        "gradient_accumulation_steps": config["training"]["gradient_accumulation_steps"],
        "seed": config["training"]["seed"],
        "lr_scheduler_type": config["training"]["lr_scheduler_type"],
        "optim": config["training"]["optim"],
        "generation_max_length": config["training"]["generation_max_length"],
    }

# 한 번의 실험 결과를 exp_log.csv에 누적 저장하는 함수
def log_experiment_result(config: dict, metrics: dict, ckpt_info: dict | None = None,
                          csv_path: str = "../outputs/exp_log.csv",) -> pd.DataFrame:

    row = {
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        **extract_exp_config(config),
    }

    # 평가지표 붙이기 (eval_* 만 필터링)
    for k, v in metrics.items():
        if k.startswith("eval_"):
            col_name = k.replace("-", "_")
            row[col_name] = v

    # checkpoint 정보가 있으면 추가
    if ckpt_info is not None:
        row.update(ckpt_info)

    # DataFrame으로 변환
    row_df = pd.DataFrame([row])

    # 기존 csv가 있으면 append, 없으면 새로 생성
    csv_path = Path(csv_path)
    if csv_path.exists():
        prev_df = pd.read_csv(csv_path)
        new_df = pd.concat([prev_df, row_df], ignore_index=True)
    else:
        new_df = row_df

    # 저장
    new_df.to_csv(csv_path, index=False, encoding="utf-8-sig")
    print(f"\n💾 실험 로그 저장 완료: {csv_path}")

    return new_df

### 모델 학습
>- 앞에서 구축한 클래스 및 함수를 활용하여 학습 진행합니다.

In [159]:
def main(config):
    # 사용할 device 정의
    device = torch.device('cuda:0' if torch.cuda.is_available()  else 'cpu')
    print(f'⚡ 사용 디바이스 : {device} | torch 버전 : {torch.__version__}')

    # 사용할 모델과 tokenizer 불러오기
    generate_model , tokenizer = load_tokenizer_and_model_for_train(config,device)
    print('✅ tokenizer 스페셜 토큰 :', tokenizer.special_tokens_map)

    # 학습에 사용할 데이터셋 불러오기
    preprocessor = Preprocess(config['tokenizer']['bos_token'], config['tokenizer']['eos_token']) # decoder_start_token: str, eos_token: str
    data_path = config['general']['data_path']
    train_inputs_dataset, val_inputs_dataset = prepare_train_dataset(config,preprocessor, data_path, tokenizer)

    # 전처리한 파일 csv로 저장하기
    save_preprocessed_csv(preprocessor, config, version="v1")       # 버전관리

    # Trainer 클래스를 불러오기
    trainer = load_trainer_for_train(config, generate_model,tokenizer,train_inputs_dataset,val_inputs_dataset)
    # 저장된 checkpoint 이후부터 실행 (중간에 학습이 중단돼도 문제 없도록 하기 위함)
    trainer.train()

    # best / last ckpt 자동 찾기
    best_ckpt, last_ckpt = get_best_and_last_checkpoints(config)

    # 두 개 체크포인트 평가 → DataFrame 비교
    all_results = []
    for name, ckpt in [("best", best_ckpt), ("last", last_ckpt)]:
        metrics = eval_checkpoint(ckpt, config, tokenizer, val_inputs_dataset)
        all_results.append({
            "type": name,
            "checkpoint": os.path.basename(ckpt),
            **{k: v for k, v in metrics.items() if k.startswith("eval_")},
        })
    df = pd.DataFrame(all_results)
    display(df.round(4))

    # 실험 로그 작성
    exp_df = log_experiment_result(
        config,
        metrics,
        ckpt_info={"which": "best", "ckpt": best_ckpt},
        csv_path="../outputs/exp_log.csv",
    )

    # wandb 종료
    wandb.finish()

In [160]:
if __name__ == "__main__":
    main(loaded_config)

⚡ 사용 디바이스 : cuda:0 | torch 버전 : 2.1.0

🤖 모델명 : digit82/kobart-summarization


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


✅ tokenizer 스페셜 토큰 : {'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask>', 'additional_special_tokens': ['#PassportNumber#', '#Person2#', '#Address#', '#Person1#', '#PhoneNumber#', '#Person3#']}

✅ 학습 데이터 로드 완료!

✅ 학습 데이터셋 생성 완료!
✅ 전처리 완료!
💾 전처리된 데이터 저장 완료 | 저장 위치: {save_dir}


Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.



✅ training arguments 정의 완료!

✅ trainer 생성 완료!


...

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.



🚀 불필요한 생성 토큰 제거 중...
PRED:  #Person2# 는 숨쉬기 힘들고 숨쉬는 것이 불편합니다. #Person1# 은 천식 
GOLD: #Person2# 는 숨쉬기 어려워합니다. 의사는 #Person2# 에게 증상을 확인하고, 천식 검사를 위해 폐 전문의에게 가볼 것을 권합니다.                                                                     

✅ 불필요한 생성 토큰 제거 완료!
{'eval_loss': 0.5232008099555969, 'eval_rouge-1': 0.31587622171998314, 'eval_rouge-2': 0.1306555021876341, 'eval_rouge-l': 0.2903306938606965, 'eval_runtime': 5.2973, 'eval_samples_per_second': 94.199, 'eval_steps_per_second': 3.02}


,type,checkpoint,eval_loss,eval_rouge-1,eval_rouge-2,eval_rouge-l,eval_runtime,eval_samples_per_second,eval_steps_per_second
0,best,checkpoint-1750,0.5180,0.3177,0.1301,0.2952,5.1245,97.376,3.122
1,last,checkpoint-2500,0.5232,0.3159,0.1307,0.2903,5.2973,94.199,3.020



💾 실험 로그 저장 완료: ../outputs/exp_log.csv


eval/loss,█▁▁▁▁▁▁▁▁▁▁▁
eval/rouge-1,▁▆▇███████▆▆
eval/rouge-2,▁▆▇▇▇███████
eval/rouge-l,▁▇▇▇▇█████▅▅
eval/runtime,█▄▄▆▅▅▅▅▅▅▁▁
eval/samples_per_second,▁▃▃▂▂▃▃▃▃▃██
eval/steps_per_second,▁▃▃▂▂▃▃▃▃▃██
train/epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇███
train/global_step,▂▂▂▂▃▃▄▄▅▅▅▅▆▆▇▇▇▇███▁▁
train/learning_rate,▁███▇▆▆▅▄▂
train/loss,█▂▁▁▁▁▁▁▁▁


### 모델 추론
>- test data를 사용하여 모델의 성능을 확인

In [161]:
# tokenization 과정까지 진행된 test 데이터 출력
def prepare_test_dataset(config,preprocessor, tokenizer):
    
    test_file_path = os.path.join(config['general']['data_path'], 'raw', 'test.csv')

    test_data = preprocessor.make_set_as_df(test_file_path,is_train=False)
    test_id = test_data['fname']
    encoder_input_test , decoder_input_test = preprocessor.make_input(test_data,is_test=True)
    print('\n✅ test 데이터 로드 완료!')

    test_tokenized_encoder_inputs = tokenizer(encoder_input_test, return_tensors="pt", padding=True,
                    add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False,)
    test_tokenized_decoder_inputs = tokenizer(decoder_input_test, return_tensors="pt", padding=True,
                    add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False,)

    test_encoder_inputs_dataset = DatasetForInference(test_tokenized_encoder_inputs, test_id, len(encoder_input_test))
    print('\n✅ test 데이터셋 생성 완료!')

    return test_data, test_encoder_inputs_dataset

In [162]:
# 추론을 위한 tokenizer와 학습시킨 모델 불러오기
def load_tokenizer_and_model_for_test(config,device):
    model_name = config['general']['model_name']
    ckt_path = config['inference']['ckt_path']
    print('\n🤖 모델명 :', model_name)

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    special_tokens_dict = {'additional_special_tokens': config['tokenizer']['special_tokens']}
    tokenizer.add_special_tokens(special_tokens_dict)

    # 자동 탐색된 best checkpoint를 모델로 불러오기
    best_ckpt, last_ckpt = get_best_and_last_checkpoints(config)
    generate_model = BartForConditionalGeneration.from_pretrained(best_ckpt)
    generate_model.resize_token_embeddings(len(tokenizer))
    generate_model.to(device)
    print('🧠 최종 모델 :', best_ckpt)
    print('✅ tokenizer & model 로드 완료!')

    return generate_model , tokenizer

In [163]:
# 학습된 모델이 생성한 요약문의 출력 결과 보기
def inference(config):
    device = torch.device('cuda:0' if torch.cuda.is_available()  else 'cpu')
    print(f'⚡ 사용 디바이스 : {device} | torch 버전 : {torch.__version__}')
    
    generate_model , tokenizer = load_tokenizer_and_model_for_test(config,device)

    data_path = config['general']['data_path']
    preprocessor = Preprocess(config['tokenizer']['bos_token'], config['tokenizer']['eos_token'])

    test_data, test_encoder_inputs_dataset = prepare_test_dataset(config,preprocessor, tokenizer)
    dataloader = DataLoader(test_encoder_inputs_dataset, batch_size=config['inference']['batch_size'])

    summary = []
    text_ids = []
    with torch.no_grad():
        for item in tqdm(dataloader):
            text_ids.extend(item['ID'])
            generated_ids = generate_model.generate(input_ids=item['input_ids'].to('cuda:0'),
                            no_repeat_ngram_size=config['inference']['no_repeat_ngram_size'],
                            early_stopping=config['inference']['early_stopping'],
                            max_length=config['inference']['generate_max_length'],
                            num_beams=config['inference']['num_beams'],
                        )
            for ids in generated_ids:
                result = tokenizer.decode(ids)
                summary.append(result)

    # 정확한 평가를 위하여 노이즈에 해당되는 스페셜 토큰 제거
    remove_tokens = config['inference']['remove_tokens']
    preprocessed_summary = summary.copy()
    for token in remove_tokens:
        preprocessed_summary = [sentence.replace(token," ") for sentence in preprocessed_summary]

    output = pd.DataFrame(
        {
            "fname": test_data['fname'],
            "summary" : preprocessed_summary,
        }
    )
    result_path = config['inference']['result_path']
    if not os.path.exists(result_path):
        os.makedirs(result_path)
    save_name = "output_preprocessed_v1.csv"                                # 버전관리
    output.to_csv(os.path.join(result_path, save_name), index=False)         
    print(f"💾 추론 결과 저장 완료 : {result_path}{save_name}")

    print("\n💬 test문 요약 결과")
    print(output)

    return output

In [164]:
# 추론 실행
if __name__ == "__main__":
    output = inference(loaded_config)

⚡ 사용 디바이스 : cuda:0 | torch 버전 : 2.1.0

🤖 모델명 : digit82/kobart-summarization


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.



🚀 체크포인트 탐색 중...
- best checkpoint : ../outputs/checkpoints/checkpoint-1750
- last_ckpt : ../outputs/checkpoints/checkpoint-2500
🧠 최종 모델 : ../outputs/checkpoints/checkpoint-1750
✅ tokenizer & model 로드 완료!

✅ test 데이터 로드 완료!

✅ test 데이터셋 생성 완료!


100%|██████████| 16/16 [00:23<00:00,  1.45s/it]

💾 추론 결과 저장 완료 : ../outputs/prediction/output_preprocessed_v1.csv

💬 test문 요약 결과
        fname                                            summary
0      test_0    Ms. Dawson은 #Person1# 에게 사내 메모를 작성하고 배포하라고 요...
1      test_1   #Person2# 는 #Person1# 에게 출퇴근 시간에 교통체증이 심하다고 말...
2      test_2    Kate는 Masha와 Hero가 이혼했다고 #Person1# 에게 말합니다. ...
3      test_3    Brian은 자신의 생일을 축하하기 위해 생일 파티에 참석했다. #Person1...
4      test_4   #Person2# 는 #Person1# 에게 올림픽 공원의 전체 좌석이 5000석...
..        ...                                                ...
494  test_495    Jack은 Charlie에게 새로운 비디오 게임을 제안하고, 그녀는 숙제를 먼저...
495  test_496   #Person2# 는 #Person1# 에게 시골 음악에 관심을 갖게 된 계기와 ...
496  test_497    Alice는 #Person1# 에게 세탁기에 비누가 들어 있지 않다고 설명하고,...
497  test_498    Steve는 Matthew에게 새로 이사할 집을 찾고 있다고 말하고, Mrs. ...
498  test_499    Frank는 Betsy에게 승진 소식을 전하고, 파티에 150명이 참석할 것이라...

[499 rows x 2 columns]


In [165]:
# 제출 전 확인
print("🔎 결측치 및 데이터타입 확인")
print("- shape:", output.shape)
print("\n- null check:\n", output.isnull().sum())
print("\n- dtypes:\n", output.dtypes)
print("\n- duplicated id:", output['fname'].duplicated().sum())

print("\n🔎요약 길이 이상치 확인")
output_summary_length = output.copy()
output_summary_length['summary_length'] = output_summary_length['summary'].str.len()
print(output_summary_length['summary_length'].describe())

🔎 결측치 및 데이터타입 확인
- shape: (499, 2)

- null check:
 fname      0
summary    0
dtype: int64

- dtypes:
 fname      object
summary    object
dtype: object

- duplicated id: 0

🔎요약 길이 이상치 확인
count    499.000000
mean     106.208417
std       19.204058
min       63.000000
25%       93.000000
50%      102.000000
75%      118.000000
max      178.000000
Name: summary_length, dtype: float64


In [166]:
# 제출 후 LB 점수 > 실험 로그에 기록
log_path = "../outputs/exp_log.csv"
exp_log = pd.read_csv(log_path)

print("📝 실험 로그")
display(exp_log)

📝 실험 로그


,timestamp,exp_name,model_name,num_train_epochs,learning_rate,per_device_train_batch_size,per_device_eval_batch_size,warmup_ratio,weight_decay,gradient_accumulation_steps,...,eval_rouge_l,eval_runtime,eval_samples_per_second,eval_steps_per_second,which,ckpt,lb_rouge_1,lb_rouge_2,lb_rouge_l,lb_final
0,2025-11-28 18:01:14,baseline_v1,digit82/kobart-summarization,20,0.00001,50,32,0.1,0.01,1,...,0.148929,5.0999,97.845,3.137,best,../outputs/checkpoints/checkpoint-1750,0.5634,0.3668,0.4819,47.0683
1,2025-12-03 10:17:28,preprocessed_v1,digit82/kobart-summarization,20,0.00001,50,32,0.1,0.01,1,...,0.284882,5.0909,98.018,3.143,best,../outputs/checkpoints/checkpoint-1750,0.5638,0.3680,0.4789,47.0228
2,2025-12-03 13:49:06,preprocessed_v1,digit82/kobart-summarization,20,0.00001,50,32,0.1,0.01,1,...,0.290331,5.2973,94.199,3.020,best,../outputs/checkpoints/checkpoint-1750,NaN,NaN,NaN,NaN


In [167]:
# # 리더보드 값 추가하기
# exp_log.loc[1, "lb_rouge_1"] = 0.5638
# exp_log.loc[1, "lb_rouge_2"] = 0.3680
# exp_log.loc[1, "lb_rouge_l"] = 0.4789
# exp_log.loc[1, "lb_final"]   = 47.0228

# exp_log.to_csv(log_path, index=False)

In [168]:
# 점수만 보기
display(exp_log[["timestamp", "exp_name", "eval_loss", "eval_rouge_1", "eval_rouge_2", "eval_rouge_l", "lb_rouge_1", "lb_rouge_2", "lb_rouge_l", "lb_final"]])

,timestamp,exp_name,eval_loss,eval_rouge_1,eval_rouge_2,eval_rouge_l,lb_rouge_1,lb_rouge_2,lb_rouge_l,lb_final
0,2025-11-28 18:01:14,baseline_v1,0.596968,0.154657,0.042543,0.148929,0.5634,0.3668,0.4819,47.0683
1,2025-12-03 10:17:28,preprocessed_v1,0.523068,0.310635,0.129385,0.284882,0.5638,0.3680,0.4789,47.0228
2,2025-12-03 13:49:06,preprocessed_v1,0.523201,0.315876,0.130656,0.290331,NaN,NaN,NaN,NaN


In [169]:
# 전처리
# 데이터 증강, hard case 분석, error pattern에 따른 rule 추가
# Solar + baseline 조합
#     (1) 에러 케이스에 한해 Solar로 후처리
#     (2) baseline 요약이 너무 짧을 때만 Solar 재요약